In [ ]:
import numpy as np
import pandas as pd

# =========================
# 0) 데이터 로드/전처리
# =========================
file_name = "mt-10yr0901-new.csv"
df = pd.read_csv(file_name, header=None)
df.columns = [
    "Date", "Open", "High", "Low", "Close",
    "Index1", "Index2", "Index3", "Index4", "Index5",
    "Index6", "Index7", "Index8", "Index9", "Index10", "Index11"
]
df["Date"] = pd.to_datetime(df["Date"]).dt.normalize()

def calculate_pct_change(df, col_name_current, col_name_previous):
    return (df[col_name_current] / df[col_name_previous].shift(1) - 1) * 100

# (시가 / 전일 종가 - 1) * 100
df["Open_Pct_Change"] = calculate_pct_change(df, "Open", "Close")
# Index6의 전일 대비 변동률
df["Index6_Pct_Change"] = calculate_pct_change(df, "Index6", "Index6")

# NaN(첫 행 등) 제거를 원하면 주석 해제
# df = df.dropna(subset=["Open_Pct_Change", "Index6_Pct_Change"]).reset_index(drop=True)

# =========================
# 1) 벡터화 준비
# =========================
index_col = "Index6_Pct_Change"
open_pct_col = "Open_Pct_Change"
open_col = "Open"
rising_rate = 2.460
falling_rate = 2.520

a = df[index_col].to_numpy(dtype=float)            # 인덱스 변동률
opct = df[open_pct_col].to_numpy(dtype=float)      # 시가 변동률
openv = df[open_col].to_numpy(dtype=float)
closev = df["Close"].to_numpy(dtype=float)

close_minus_open = closev - openv
open_minus_close = openv - closev

# 미리 계산해두는 비교 기준 (각 시나리오에서 사용)
adj_rising = a * rising_rate
adj_falling = a * falling_rate

# =========================
# 2) 각 필터 값에 대한 수익을 벡터로 계산하는 함수들
#    (기존 엄격부등호 그대로 유지)
# =========================
def profit_rising_min(filter_value: float) -> float:
    # 상승 상황 & (index > filter_min)  [엄격 >]
    mask = (a > 0) & (a > filter_value)
    # if open_pct > adj_rising -> + (Close-Open)
    # elif open_pct < adj_rising -> + (Open-Close)
    cmp = opct - adj_rising
    contrib = np.where(cmp > 0, close_minus_open, np.where(cmp < 0, open_minus_close, 0.0))
    return float(np.sum(np.where(mask, contrib, 0.0)))

def profit_rising_max(filter_value: float) -> float:
    # 상승 상황 & (index < filter_max)  [엄격 <]
    mask = (a > 0) & (a < filter_value)
    cmp = opct - adj_rising
    contrib = np.where(cmp > 0, close_minus_open, np.where(cmp < 0, open_minus_close, 0.0))
    return float(np.sum(np.where(mask, contrib, 0.0)))

def profit_falling_min(filter_value: float) -> float:
    # 하락 상황 & (index > filter_min)  [엄격 >]  (filter_value는 음수 구간)
    mask = (a < 0) & (a > filter_value)
    # if open_pct < adj_falling -> + (Open-Close)
    # elif open_pct > adj_falling -> + (Close-Open)
    cmp = opct - adj_falling
    contrib = np.where(cmp < 0, open_minus_close, np.where(cmp > 0, close_minus_open, 0.0))
    return float(np.sum(np.where(mask, contrib, 0.0)))

def profit_falling_max(filter_value: float) -> float:
    # 하락 상황 & (index < filter_max)  [엄격 <]
    mask = (a < 0) & (a < filter_value)
    cmp = opct - adj_falling
    contrib = np.where(cmp < 0, open_minus_close, np.where(cmp > 0, close_minus_open, 0.0))
    return float(np.sum(np.where(mask, contrib, 0.0)))

# =========================
# 3) 최적 필터 탐색 (간극 0.01 유지)
# =========================
def argmax_with_range(fn, start, stop, step):
    best_val = -np.inf
    best_arg = None
    # np.arange 간극 그대로 유지
    vals = np.arange(start, stop, step)
    for v in vals:
        p = fn(float(v))
        if p > best_val:
            best_val = p
            best_arg = float(v)
    return best_arg, best_val

# 기존 범위/간극 그대로
rising_filter_min, profit_r_min = argmax_with_range(profit_rising_min, 0.0, 0.5, 0.01)
rising_filter_max, profit_r_max = argmax_with_range(profit_rising_max, 0.5, 20.0, 0.01)
falling_filter_min, profit_f_min = argmax_with_range(profit_falling_min, -20.0, -0.5, 0.01)
falling_filter_max, profit_f_max = argmax_with_range(profit_falling_max, -0.5, 0.0, 0.01)

# =========================
# 4) 최적 필터값으로 다시 각 수익 계산 (검증용)
# =========================
final_profit_r_min = profit_rising_min(rising_filter_min)
final_profit_r_max = profit_rising_max(rising_filter_max)
final_profit_f_min = profit_falling_min(falling_filter_min)
final_profit_f_max = profit_falling_max(falling_filter_max)

print(f"상승 상황 rising_filter_min 수익: {final_profit_r_min:.2f}")
print(f"상승 상황 rising_filter_max 수익: {final_profit_r_max:.2f}")
print(f"하락 상황 falling_filter_min 수익: {final_profit_f_min:.2f}")
print(f"하락 상황 falling_filter_max 수익: {final_profit_f_max:.2f}")

print(
    f"최적 필터값 -> "
    f"Rising Min = {rising_filter_min:.2f}, "
    f"Rising Max = {rising_filter_max:.2f}, "
    f"Falling Min = {falling_filter_min:.2f}, "
    f"Falling Max = {falling_filter_max:.2f}"
)


: 